In [ ]:
from rdkit import Chem,RDLogger
from rdkit.Chem import Descriptors
import pandas as pd
import numpy as np
import ast,copy,itertools
from collections import Counter
from itertools import combinations,product


# 1) Get RXNMapping
## 1A) Use Paper Mappings
'RXN_MappingList.txt' was in a format where each line is a SMILES and the proceding line the atom mapping as a list. An example of this format is:
C=CC=C.C=C>>C1=CCCCC1

[1,2,3,4,6,5] 


The index of this list is the atom mapping number in the product but the value is the atom mapping number in the reactant.
To convert: 

Reactant > product ls.index(atommapping)

Product > reactant ls[atommapping]

Note that SMILES results  from RXNMapper will be 1 indexed whereas the list extracted will be 0 indexed. The 1 indexed method (1st atom has mapping of 1) should be used in general

In [ ]:
### Previous data redone to be mapped
with open('RXN_MappingList.txt','r') as rf:
    original_rxn_ls = rf.readlines()
original_rxn_ls = [x for x in zip(original_rxn_ls[::2],original_rxn_ls[1::2])]
original_rxn_ls = [[x[0].strip(),ast.literal_eval(x[1])] for x in original_rxn_ls]

def addmap(smiles,mapping):
    rsmiles,psmiles = smiles.split('>>')
    rsmiles = '.'.join(rsmiles.split('.')[:2])
    rmol = Chem.MolFromSmiles(rsmiles)
    pmol = Chem.MolFromSmiles(psmiles)
    if abs(Descriptors.MolWt(rmol) - Descriptors.MolWt(pmol))>0.1:
        return False
    if 0 not in mapping:
        return False
    for atom in rmol.GetAtoms():
        atom.SetAtomMapNum(mapping.index(atom.GetIdx())+1)
    for atom in pmol.GetAtoms():
        atom.SetAtomMapNum(atom.GetIdx()+1)
    return f'{Chem.MolToSmiles(rmol)}>>{Chem.MolToSmiles(pmol)}'

rxn_ls = [addmap(*x) for x in original_rxn_ls]
rxn_ls = [x for x in rxn_ls if x] ### ignore if using above
rxn_ls


## 1B) Generate Your Own Mappings
The results can be then be reformatted to the same format as in 1A. This can be done by iterating through the product atoms and adding the atommapping to a list.

In [ ]:
### Run this in Python console

# from rxnmapper import BatchedMapper
# import pandas as pd
# rxn_mapper = BatchedMapper(batch_size=32)
# df = pd.read_csv('file.csv')
# rxns = list(df['Cleaned_Reaction_Smiles']) ### Get the reaction smiles
# results = list(rxn_mapper.map_reactions(rxns))
# with open('outputmapping.txt','w') as wf:
#     for r in results:
#         wf.write(f'{r}\n')

# 2) Extract Reaction Centres


In [ ]:
def rm_map_and_clean_smiles(smiles):
    my_mol = Chem.MolFromSmiles(smiles)
    atomMap = "molAtomMapNumber"
    for atom in my_mol.GetAtoms():
        if atom.HasProp(atomMap):
                atom.ClearProp(atomMap)
    return Chem.MolToSmiles(Chem.MolFromSmiles(Chem.MolToSmiles(my_mol)))

def getmols(mapsmiles):
    precursors, products = mapsmiles.split(">>")
    precursors_mol = Chem.MolFromSmiles(precursors)
    products_mol = Chem.MolFromSmiles(products)
    return precursors_mol,products_mol

def get_changing_atoms(precursors_mol,products_mol):
    changingatomsmap = []
    changingatomspreidx = []
    changingatomsprodidx = []
    for atom in products_mol.GetAtoms():
        prod_atom_neighbors = sorted([x.GetAtomMapNum() for x in atom.GetNeighbors()])
        preatomidx = [x.GetAtomMapNum() for x in precursors_mol.GetAtoms()].index(atom.GetAtomMapNum())
        try:
            ### For each atom in the reactant/precursor, get the neighbouring atom indexes
            ### We take the atom neighbours from the reactant and convert them into the corresponding product index
            pre_atom_neighbors = sorted([x.GetAtomMapNum() for x in precursors_mol.GetAtomWithIdx(preatomidx).GetNeighbors()])
        except:
            continue
        if set(prod_atom_neighbors) != set(pre_atom_neighbors):
            ### see if the neighbouring atoms differs between reactants and products. AKA check for atoms which change bonding
            changingatomsmap.append(atom.GetAtomMapNum())
            changingatomspreidx.append(preatomidx)
            changingatomsprodidx.append(atom.GetIdx())
    return changingatomsmap,changingatomspreidx,changingatomsprodidx

def get_dedp(precursors_mol,products_mol,changingatomspreidx):
    forming_bonds = []
    dp_atoms = []
    bondls = [c for c in combinations(changingatomspreidx, 2)]
    bondls = [x for x in bondls if precursors_mol.GetBondBetweenAtoms(x[0],x[1])!=None]
    if len(bondls) == 1:
        dp_atoms_preidx = list(bondls[0])
    else:
        return False
    de_atoms_preidx = list(set(changingatomspreidx)-set(dp_atoms_preidx))
    if len(de_atoms_preidx)!=2:
        return False
    ### Get Mappings and product indexes for later use
    de_atoms_map = [precursors_mol.GetAtomWithIdx(x).GetAtomMapNum() for x in de_atoms_preidx]
    dp_atoms_map = [precursors_mol.GetAtomWithIdx(x).GetAtomMapNum() for x in dp_atoms_preidx]
    prodmap = [x.GetAtomMapNum() for x in products_mol.GetAtoms()]
    de_atoms_prodidx = [prodmap.index(x) for x in de_atoms_map]
    dp_atoms_prodidx = [prodmap.index(x) for x in dp_atoms_map]

    return [de_atoms_map,dp_atoms_map],[de_atoms_preidx,dp_atoms_preidx],[de_atoms_prodidx,dp_atoms_prodidx]

def get_full_rc(precursors_mol,products_mol,de_atoms_map,dp_atoms_map,de_atoms_preidx,dp_atoms_preidx,de_atoms_prodidx,dp_atoms_prodidx):
    reaction_centres =[]
    atom0neighbours = [x.GetIdx() for x in precursors_mol.GetAtomWithIdx(de_atoms_preidx[0]).GetNeighbors()]
    atom1neighbours = [x.GetIdx() for x in precursors_mol.GetAtomWithIdx(de_atoms_preidx[1]).GetNeighbors()]
    bondcombos = [x for x in product(atom0neighbours,atom1neighbours)]
    diene_preidx = [[de_atoms_preidx[0],x[0],x[1],de_atoms_preidx[1]] for x in bondcombos if precursors_mol.GetBondBetweenAtoms(x[0],x[1])!=None]
    ### Get mappings and product indexes for later use
    diene_map = [[precursors_mol.GetAtomWithIdx(y).GetAtomMapNum() for y in x] for x in diene_preidx]
    prodmap = [x.GetAtomMapNum() for x in products_mol.GetAtoms()]
    diene_prodidx = [[prodmap.index(y) for y in x] for x in diene_map]
    
    fullrc_map = []
    fullrc_preidx = []
    fullrc_prodidx = []
    
    for n,diene in enumerate(diene_prodidx):
        if products_mol.GetBondBetweenAtoms(diene[-1],dp_atoms_prodidx[0])!=None and products_mol.GetBondBetweenAtoms(diene[0],dp_atoms_prodidx[1])!=None:
            fullrc_map.append(diene_map[n] + dp_atoms_map)
            fullrc_preidx.append(diene_preidx[n] + dp_atoms_preidx)
            fullrc_prodidx.append(diene_prodidx[n] + dp_atoms_prodidx)
        elif products_mol.GetBondBetweenAtoms(diene[-1],dp_atoms_prodidx[1])!=None and products_mol.GetBondBetweenAtoms(diene[0],dp_atoms_prodidx[0])!=None:
            fullrc_map.append(diene_map[n] + list(reversed(dp_atoms_map)))
            fullrc_preidx.append(diene_preidx[n] + list(reversed(dp_atoms_preidx)))
            fullrc_prodidx.append(diene_prodidx[n] + list(reversed(dp_atoms_prodidx)))

    return fullrc_map, fullrc_preidx, fullrc_prodidx
   
def verify_rc_bonding(precursors_mol,fullrc_map, fullrc_preidx, fullrc_prodidx):
    verified_rc_map = []
    verified_rc_preidx = []
    verified_rc_prodidx = []
    for n,rc in enumerate(fullrc_preidx):
        bond01 = str(precursors_mol.GetBondBetweenAtoms(rc[0],rc[1]).GetBondType())
        if bond01 != 'AROMATIC' and bond01 != 'DOUBLE':
            continue
        bond12 = str(precursors_mol.GetBondBetweenAtoms(rc[1],rc[2]).GetBondType())
        if bond12 != 'AROMATIC' and bond12 != 'SINGLE':
            continue
        bond23 = str(precursors_mol.GetBondBetweenAtoms(rc[2],rc[3]).GetBondType())
        if bond23 != 'AROMATIC' and bond23 != 'DOUBLE':
            continue
        if precursors_mol.GetBondBetweenAtoms(rc[3],rc[4]) != None:
            continue
        bond45 = str(precursors_mol.GetBondBetweenAtoms(rc[4],rc[5]).GetBondType())
        if  bond45 != 'AROMATIC' and bond45 != 'DOUBLE' and bond45 != 'TRIPLE':
            continue
        if precursors_mol.GetBondBetweenAtoms(rc[5],rc[0]) != None:
            continue
        if len(set(rc)) ==6 and len(set(fullrc_map[n])) ==6 and len(set(fullrc_prodidx[n])) ==6: 
            verified_rc_preidx.append(rc)
            verified_rc_map.append(fullrc_map[n])
            verified_rc_prodidx.append(fullrc_prodidx[n])
            
    return verified_rc_map, verified_rc_preidx, verified_rc_prodidx

def rc6to4(verified_rc_map, verified_rc_preidx, verified_rc_prodidx):
    if len(verified_rc_map) ==1:
        return verified_rc_map[0], verified_rc_preidx[0], verified_rc_prodidx[0]
    reduced_rcs_map = [tuple([x[0],x[3],x[4],x[5]]) for x in verified_rc_map]
    reduced_rcs_preidx = [tuple([x[0],x[3],x[4],x[5]]) for x in verified_rc_preidx]
    reduced_rcs_prodidx = [tuple([x[0],x[3],x[4],x[5]]) for x in verified_rc_prodidx]
    if len(set(reduced_rcs_map)) ==1:
        return reduced_rcs_map[0], reduced_rcs_preidx[0], reduced_rcs_prodidx[0]
    else:
        return None, None, None


atommapdic = {}
for n,rxn in enumerate(rxn_ls):
    print(f'{n}/{len(rxn_ls)}',end='\r')
    rxn = rxn.strip()
    if len(rxn) <4:
        continue
    precursors_mol,products_mol = getmols(rxn)
    changingatomsmap,changingatomspreidx,changingatomsprodidx = get_changing_atoms(precursors_mol,products_mol)
    dedpresults = get_dedp(precursors_mol,products_mol,changingatomspreidx)
    if dedpresults!= False:
        [de_atoms_map,dp_atoms_map],[de_atoms_preidx,dp_atoms_preidx],[de_atoms_prodidx,dp_atoms_prodidx] = dedpresults
    else:
        atommapdic[rxn] = [f"""{rm_map_and_clean_smiles(rxn.split('>>')[0])}>>{rm_map_and_clean_smiles(rxn.split('>>')[1])}""",None, None, None]
        continue
    fullrc_map, fullrc_preidx, fullrc_prodidx = get_full_rc(precursors_mol,products_mol,de_atoms_map,dp_atoms_map,de_atoms_preidx,dp_atoms_preidx,de_atoms_prodidx,dp_atoms_prodidx)
    verified_rc_map, verified_rc_preidx, verified_rc_prodidx = verify_rc_bonding(precursors_mol,fullrc_map, fullrc_preidx, fullrc_prodidx)
    verified_rc_map, verified_rc_preidx, verified_rc_prodidx = rc6to4(verified_rc_map, verified_rc_preidx, verified_rc_prodidx)
    atommapdic[rxn] = [f"""{rm_map_and_clean_smiles(rxn.split('>>')[0])}>>{rm_map_and_clean_smiles(rxn.split('>>')[1])}""",verified_rc_map, verified_rc_preidx, verified_rc_prodidx]



# 3) Ring Check
Check for the formation of a 6 membered ring, used to exclude tautomerization and other invalid reactions.

In [ ]:
def ring_change_6mem(rsmiles,psmiles):
    ### Does not work for bicyclic
    rmol = Chem.MolFromSmiles(rsmiles,sanitize=False)
    pmol = Chem.MolFromSmiles(psmiles,sanitize=False)
    Chem.SanitizeMol(rmol,Chem.SanitizeFlags.SANITIZE_FINDRADICALS|Chem.SanitizeFlags.SANITIZE_SETAROMATICITY|Chem.SanitizeFlags.SANITIZE_SETCONJUGATION|Chem.SanitizeFlags.SANITIZE_SETHYBRIDIZATION|Chem.SanitizeFlags.SANITIZE_SYMMRINGS) # Chem.SanitizeFlags.SANITIZE_KEKULIZE|
    Chem.SanitizeMol(pmol,Chem.SanitizeFlags.SANITIZE_FINDRADICALS|Chem.SanitizeFlags.SANITIZE_SETAROMATICITY|Chem.SanitizeFlags.SANITIZE_SETCONJUGATION|Chem.SanitizeFlags.SANITIZE_SETHYBRIDIZATION|Chem.SanitizeFlags.SANITIZE_SYMMRINGS) # Chem.SanitizeFlags.SANITIZE_KEKULIZE|
    rmol_rings = [x for x in rmol.GetRingInfo().AtomRings()]
    rmol_rings += get_bicyclic_6mem(rmol_rings,rmol)
    rmol_rings = redundant_rings(rmol_rings)
    pmol_rings = [x for x in pmol.GetRingInfo().AtomRings()]
    pmol_rings += get_bicyclic_6mem(pmol_rings,pmol)
    pmol_rings = redundant_rings(pmol_rings)    
    
    return len([x for x in pmol_rings if len(x)==6]) - len([x for x in rmol_rings if len(x)==6])
    

def get_bicyclic_6mem(rings,mol):
    """ Check for 6membered rings that are formed from bicyclic systems. RDKit does not find these"""
    ### Find systems rings which share 3 or more atoms, indicating they have shared briding atoms
    rings_3shard = []
    for r in rings:
        share_rings = [x for x in rings if len(set(r)-set(x))<3 and len(set(r)-set(x))!=0]
        if len(share_rings)>0:
            for x in share_rings:
                if [list(x),list(r)] in rings_3shard:
                    continue
                rings_3shard.append([list(r),list(x)])
    additional_rings = []
    ### Get the indexes for the bridging atoms and the atoms on either side of the bridge
    for r in rings_3shard:
        ring1atoms = list(set(r[0])-set(r[1]))
        ring2atoms = list(set(r[1])-set(r[0]))
        tripointatoms = [x.GetIdx() for x in mol.GetAtoms() if len(x.GetNeighbors())>2 and x.GetIdx() in r[0]]
        ### Generate all possible rings (8)
        ring_possibilities = list([x for x in itertools.product(*[ring1atoms,ring1atoms,tripointatoms,ring2atoms,ring2atoms,tripointatoms]) if len(set(x))==6])
        ### Check if the rings are valid by ensuring each atom is sequentially bonded to another in the ring
        foundring = True
        invalid_rings = []
        for ring in ring_possibilities:
            for n,idx in enumerate(ring):
                if mol.GetBondBetweenAtoms(idx,ring[n-1]):
                    pass
                else:
                    invalid_rings.append(ring)
                    break
        additional_rings += [x for x in ring_possibilities if x not in invalid_rings]
    if len(additional_rings) == 0:
        return []
    additional_rings = [x for x in redundant_rings(additional_rings) if len(x)==len(set(x))]
    return additional_rings
    
def redundant_rings(new_rings):
    ### Ensure that the new rings are unique, instances will occur where two different orderings encode the same ring
    ## This must be prevented so the rings are checked forwards and backwards to see if the relative ordering is identical
    new_rings_unique = []
    for n,ring_combo in enumerate(new_rings):
        if n==0:
            new_rings_unique.append(list(ring_combo))
            continue
        if new_rings[0][0] not in ring_combo:
            new_rings_unique.append(list(ring_combo))
            continue
        srt_idx = ring_combo.index(new_rings[0][0])
        new_combo = ring_combo[srt_idx:] + ring_combo[:srt_idx]
        ### Reverse the ordering and try again. To test if the relative ordering matches, we must try combinations
        ## to get an absolute ordering that matches.
        if new_combo not in new_rings_unique:
            ring_combo_rev = [x for x in reversed(ring_combo)]
            srt_idx = ring_combo_rev.index(new_rings[0][0])
            new_combo = ring_combo_rev[srt_idx:] + ring_combo_rev[:srt_idx]
            if new_combo not in new_rings_unique:
                new_rings_unique.append(new_combo)
            else:
                pass
        else:
            pass
    return new_rings_unique

def getmapping(mapsmiles):
    rsmiles,psmiles = mapsmiles.split('>>')
    pmol = Chem.MolFromSmiles(psmiles)
    return [x.GetAtomMapNum() for x in pmol.GetAtoms()]


In [ ]:
# verified_rcs = {v[0]:[v[1],getmapping(k)] for k,v in atommapdic_noNone.items()}
verified_rcs = {v[0]:[v[1],getmapping(k)] for k,v in atommapdic.items()}
verified_rcs_ring = {k.strip():v for k,v in verified_rcs.items() if ring_change_6mem(k.split('>>')[0],k.split('>>')[1]) >0}


# 4) Use Verified RXNs to Edit DataFrames

In [ ]:
def checkmembership(element,ls):
    if element in ls:
        return True
    return False

def clean_smiles(smiles):
    return Chem.MolToSmiles(Chem.MolFromSmiles(smiles))

def clean_rxn(rxnsmiles):
    if type(rxnsmiles)!=str:
        return rxnsmiles
    reactants,products = rxnsmiles.split('>>')
    return f"""{Chem.MolToSmiles(Chem.MolFromSmiles(reactants))}>>{Chem.MolToSmiles(Chem.MolFromSmiles(products))}"""
    
def attempt_match(rxn,dic):
    try:
        return dic[rxn]
    except:
        return np.nan

In [ ]:
df_verifiedrxns = pd.read_csv(f'unverifiedrxns.csv') ### File from Gryzbowski_Data_Acquisition.ipynb
df_verifiedrxns['Cleaned_Reaction_Smiles'] = df_verifiedrxns.apply (lambda row: clean_rxn(row['Cleaned_Reaction_Smiles']), axis=1)

df_verifiedrxns = df_verifiedrxns[df_verifiedrxns['Cleaned_Reaction_Smiles'].isin(verified_rcs_ring.keys())]
df_verifiedrxns.to_csv(f'finaldataset_{df_verifiedrxns.shape[0]}.csv')


### Manual editing is needed for the following IDs: 1449060,1449130,1440357, and 47405237
### 9 Additional non-Reaxys reactions were donated to us.